In [ ]:
pip install --upgrade pip

In [ ]:
!pip install pandas
!pip install openpyxl

In [11]:
import pandas as pd
from IPython.display import display, Markdown
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows (use with caution)
pd.set_option('display.max_colwidth', None) # Show full content of each column
pd.set_option('expand_frame_repr', False)   # Prevent line wrapping

## Add your statement to current working directory in csv format and rename it to statement.csv

In [ ]:
# Load statement CSV
file_path = 'statement.csv'
df = pd.read_csv(file_path)
display(df.head)

## Data Cleanup Phase
- **Clean up your statement  as it may contain some rows on top and bottom which may contain some meta data like your name, statement date, your account number etc.**

In [ ]:
# Remove the first 19 rows which may contain metadata
df = df.iloc[19:]

# Set the first row as the header
df.columns = df.iloc[0]

# Remove the header row from the DataFrame
df = df.iloc[1:]

# Reset the index of the DataFrame
df = df[1:].reset_index(drop=True)

# Keep only the 4th and 1st columns
df = df.iloc[:, [4, 1]]

# Define column names for amount and description
amount_col = 'amount'
description_col = 'description'

# Rename columns "Narration" to "description" and "Withdrawal Amt." to "amount"
df = df.rename(columns={"Narration": description_col, "Withdrawal Amt.": amount_col})

# Display the first 5 rows of the DataFrame
display(df.head(5))

# Display the column names of the DataFrame
display(df.columns)

## Update tobacco_sellers with your regular pan walas

In [ ]:
# Check that meta data has been removed and column names are correct.
display(df[description_col].head(3))
display(df[amount_col].head(3))

### Create a file named **tobacco_sellers.txt** with the following content:
Name 1 <br>
Name 2 <br>
Name 3 <br>

In [ ]:
# with open('tobacco_sellers.txt', 'r') as file:
#     tobacco_sellers = [line.strip() for line in file.readlines()]
    
def read_file_to_array(file_name):
    """
    Reads a file and returns a list containing each line as an element.
    
    :param file_name: str - Name of the file to read
    :return: list - List of lines from the file without newline characters
    """
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            return [line.strip() for line in file.readlines()]
    except FileNotFoundError:
        print(f"Error: File '{file_name}' not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage:
# tobacco_sellers = read_file_to_array('tobacco_sellers.txt')
# print(tobacco_sellers)
tobacco_sellers = read_file_to_array('tobacco_sellers.txt')
print(tobacco_sellers)

tobacco_keyword = "pan wala"

In [ ]:
# Replace occurrences of tobacco-related keywords in narration with "pan wala" (case insensitive)
def replace_tobacco_sellers(text):
    text_lower = text.lower()
    for seller in tobacco_sellers:
        if seller.lower() in text_lower:
            text_lower = text_lower.replace(seller.lower(), "pan wala")
    return text_lower

df[description_col] = df[description_col].astype(str).apply(replace_tobacco_sellers)
display(df.head(5))

### We might need to remove some transactions later on for items matching "pan wala" somehow in the list.
#### Create a file named **valid_non_tobacco_sellers.txt** with the following content:
Name 1 <br>
Name 2 <br>
Name 3 <br>

In [ ]:

# Ensure column names are lowercase for consistency
df.columns = df.columns.str.lower()


if amount_col in df.columns:
    df = df.dropna(subset=[amount_col])
    df[amount_col] = pd.to_numeric(df[amount_col], errors='coerce')
    
# Define removable items
removable_items = read_file_to_array('valid_non_tobacco_sellers.txt')

# Remove items containing any of the removable items
df = df[~df.apply(lambda row: row.astype(str).str.contains('|'.join(removable_items), case=False, na=False).any(), axis=1)]

def contains_tobacco(row):
    return any(keyword in str(row.get(description_col, "")).lower() for keyword in [tobacco_keyword])

tobacco_spends = df[df.apply(contains_tobacco, axis=1)]
display(tobacco_spends.head(2))
# Calculate total amount spent
total_spent = tobacco_spends[amount_col].sum()
display(Markdown(f"# Total amount spent on tobacco-related purchases: {total_spent}"))

In [19]:
tobacco_spends.to_csv('tobacco_spends.csv', index=False)

### Your tobacco_spends are saved to tobacco_spends.csv